In [1]:
import pandas as pd
import numpy as np

In [2]:
movie_df = pd.read_csv("movies.csv")
rating_df = pd.read_csv("ratings.csv")


In [3]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
df = pd.merge(movie_df,rating_df,on='movieId')
df.drop(['genres','timestamp'],axis=1,inplace=True)
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [6]:
new_df = df.dropna(subset=['title'],axis=0)
total_rating_count = (new_df.groupby(by=['title'])['rating'].count().
                      reset_index().rename(columns = {'rating': 'TotalRatingCount'})
                     )
total_rating_count.head()

,title,TotalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [7]:
new_df_withTotalRatingCount = pd.merge(new_df,total_rating_count,on='title')
new_df_withTotalRatingCount.head()

,movieId,title,userId,rating,TotalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [8]:
print(total_rating_count['TotalRatingCount'].describe())

count    9719.000000
mean       10.375141
std        22.406220
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       329.000000
Name: TotalRatingCount, dtype: float64


In [9]:
popular_movies = new_df_withTotalRatingCount.query('TotalRatingCount >= 50')
popular_movies.head()

,movieId,title,userId,rating,TotalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [10]:
movie_rating_perUser = popular_movies.pivot_table(index = 'title',
                                                  columns = 'userId',values='rating').fillna(0)
movie_rating_perUser.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [11]:
from scipy.sparse import csr_matrix

movie_rating_perUser_matrix = csr_matrix(movie_rating_perUser.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_rating_perUser_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [12]:
query_index = np.random.choice(movie_rating_perUser.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_rating_perUser.iloc[query_index,:].values.reshape(1,-1), n_neighbors = 5)

243


In [13]:
movie_rating_perUser.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [14]:
for i in range(0,len(distances.flatten())):
    if i == 0:
        print('Recommended for {0}: \n'.format(movie_rating_perUser.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}'.format(i,movie_rating_perUser.index[indices.flatten()[i]], distances.flatten()[i]))

Recommended for Lethal Weapon (1987): 

1: RoboCop (1987), with distance of 0.3374175486826607
2: Indiana Jones and the Temple of Doom (1984), with distance of 0.3954482158424708
3: Ghostbusters (a.k.a. Ghost Busters) (1984), with distance of 0.43907330075962603
4: Die Hard (1988), with distance of 0.45359737292893365
